In [1]:
import pandas as pd
import warnings
import glob
import numpy as np


# Configurações e avisos
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# Definir colunas necessárias
colunas_necessarias = ['TIPO_PRODUTO','USU_NOME','DEV_CPF','ACN_DESCRICAO','DEA_DATA','ATRASO']

# Definir caminho para os arquivos
caminho = r"R:\TI\MIS SUP\ACIONAMENTOS_CLARO_NET_GEVENUE\INTERSIC"
arquivos = glob.glob(caminho + "/*.txt")

# Inicializar uma lista para armazenar DataFrames
dfList = []

# Ler e processar cada arquivo
for arquivo in arquivos:
    # Ler o arquivo inteiro de uma vez
    df = pd.read_csv(arquivo, encoding='iso-8859-1', sep=";",error_bad_lines=False, usecols=colunas_necessarias,dtype={'DEV_CPF':'str'})
    # Adicionar o DataFrame à lista
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
acionamento_intersic = pd.concat(dfList, sort=False, ignore_index=True)

# filtrando oq é atn
acionamento_intersic = acionamento_intersic[acionamento_intersic['USU_NOME'].str.contains('ATN')]
# Remover o conteúdo entre parênteses na coluna 'Negociador'
acionamento_intersic['USU_NOME'] = acionamento_intersic['USU_NOME'].str.replace(r'\s*\([^)]*\)', '', regex=True)

acionamento_intersic['DEA_DATA'] = acionamento_intersic['DEA_DATA'].str[:10]


# Criando coluna Faixa de atraso com suas respectivas faixas
acionamento_intersic['ATRASO'] = acionamento_intersic['ATRASO'].astype('int64')
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

## Ajustando layout
acionamento_intersic = acionamento_intersic.drop(columns=['ATRASO'])[['DEA_DATA','TIPO_PRODUTO',	'USU_NOME',	'DEV_CPF',	'ACN_DESCRICAO','Faixa_de_Atraso']]


In [2]:
# Configurações e avisos
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# Definir colunas necessárias
colunas_necessarias = ['Nome do UsuÃ¡rio','CPF/CNPJ','Aging','Data do Acionamento','Acionamento Feito','Valor Net','Valor Embratel','Valor Movel','Valor Claro TV','Valor Claro Fixo']

# Definir caminho para os arquivos
caminho = r"R:\TI\MIS SUP\ACIONAMENTOS_CLARO_NET_GEVENUE\GEVENUE"
arquivos = glob.glob(caminho + "/*.csv")

# Inicializar uma lista para armazenar DataFrames
dfList = []

# Ler e processar cada arquivo
for arquivo in arquivos:
    # Ler o arquivo inteiro de uma vez
    df = pd.read_csv(arquivo, encoding='iso-8859-1', sep="|",error_bad_lines=False, usecols=colunas_necessarias)
    # Adicionar o DataFrame à lista
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
acionamento_gevenue = pd.concat(dfList, sort=False, ignore_index=True)

## Remover a aspas do cpf
acionamento_gevenue['CPF/CNPJ'] = acionamento_gevenue['CPF/CNPJ'].str.replace(r'\D', '', regex=True)


# Função para determinar os produtos em uma lista
def determinar_produto(row):
    produtos = []
    if pd.notna(row['Valor Net']):
        produtos.append('GEVENUE NET')
    if pd.notna(row['Valor Embratel']):
        produtos.append('GEVENUE NET')
    if pd.notna(row['Valor Movel']):
        produtos.append('GEVENUE CLARO MOVEL')
    if pd.notna(row['Valor Claro TV']):
        produtos.append('GEVENUE CLARO TV')
    if pd.notna(row['Valor Claro Fixo']):
        produtos.append('GEVENUE CLARO FIXO')
    return produtos if produtos else None

# Aplicar a função ao DataFrame
acionamento_gevenue['PRODUTO'] = acionamento_gevenue.apply(determinar_produto, axis=1)

# Explodir a coluna PRODUTO para criar linhas separadas
acionamento_gevenue = acionamento_gevenue.explode('PRODUTO')

## Removendo duplicatas para nao trazer o produto net duas vezes desnecessáriamente
acionamento_gevenue = acionamento_gevenue.drop_duplicates(['Data do Acionamento','CPF/CNPJ','PRODUTO']).reset_index(drop=True)

# Criando coluna Faixa de atraso com suas respectivas faixas
acionamento_gevenue['Aging'] = acionamento_gevenue['Aging'].astype('int64')
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

### ajustando layout
acionamento_gevenue = acionamento_gevenue.rename(columns={'Nome do UsuÃ¡rio':'USU_NOME','CPF/CNPJ':'DEV_CPF','Data do Acionamento':'DEA_DATA','Acionamento Feito':'ACN_DESCRICAO','PRODUTO':'TIPO_PRODUTO'})\
[['DEA_DATA','TIPO_PRODUTO',	'USU_NOME',	'DEV_CPF',	'ACN_DESCRICAO','Faixa_de_Atraso']]
acionamento_gevenue


,DEA_DATA,TIPO_PRODUTO,USU_NOME,DEV_CPF,ACN_DESCRICAO,Faixa_de_Atraso
0,01/07/2024,GEVENUE NET,"VANESSA, DA COSTA LOPES",51623463572,Boleto - Envio do Boleto por Email,Atraso (361 a 1080)
1,01/07/2024,GEVENUE CLARO MOVEL,"VANESSA, DA COSTA LOPES",51623463572,Boleto - Envio do Boleto por Email,Atraso (361 a 1080)
2,01/07/2024,GEVENUE NET,"VANESSA, DA COSTA LOPES",27950755806,Boleto - Envio do Boleto por Email,Atraso (91 a 360)
3,01/07/2024,GEVENUE CLARO MOVEL,"VANESSA, DA COSTA LOPES",27950755806,Boleto - Envio do Boleto por Email,Atraso (91 a 360)
4,01/07/2024,GEVENUE NET,"IZABEL, CRISTINA ALMEIDA",05290041000183,Boleto - Envio do Boleto por Email,Atraso (1081 a 1800)
...,...,...,...,...,...,...
82772,11/07/2024,GEVENUE NET,FELIPE DIAS ALVES LUIS,16942531472,Acionamento Ext. - Queda de LigaÃ§Ã£o,Atraso (1081 a 1800)
82773,11/07/2024,GEVENUE CLARO MOVEL,NASCIMENTO VIANA RAMON,49500422115,Acionamento Ext. - Telefone Errado,Atraso (361 a 1080)
82774,11/07/2024,GEVENUE NET,CRISTINA ASSUMPCAO DA PENHA RENATA,26513048591,Acionamento Ext. - Telefone Errado,Atraso (1081 a 1800)
82775,11/07/2024,GEVENUE CLARO MOVEL,MOTA DA SILVA GABRIEL,02737398673,InformaÃ§Ãµes de contato - Adicionado,Atraso (361 a 1080)


In [6]:
acionamentos_teles = pd.concat([acionamento_intersic,acionamento_gevenue])
acionamentos_teles = acionamentos_teles.assign(ACIONAMENTO = 1)

lista_cpc = ['ALEGA PROCESSO JUDICIAL',
'ALEGA DESEMPREGO',
'ALEGA FRAUDE',
'ESTÁ NEGOCIANDO COM O CREDOR',
'REENVIO DE BOLETO',
'CONTESTAÇÃO',
'ALEGA PAGAMENTO',
'NÃO RECONHECE A LINHA',
'PREVENTIVO REALIZADO',
'PROMESSA DE PAGAMENTO',
'SEM CONDIÇÕES FINANCEIRAS',
'NÃO CONCORDA COM NEGATIVAÇÃO SPC/SERASA',
'NÃO CONCORDA VALOR COBRADO',
'PROBLEMAS DE SAÚDE',
'RECUSA-SE A PAGAR',
'Acionamento Ext. - Alega Desemprego',
'Acionamento Ext. - Alega Fraude',
'Acionamento Ext. - NÃ£o concorda com valor cobrado',
'Acionamento Ext. - NÃ£o reconhece a linha',
'Acionamento Ext. - Preventivo realizado',
'Acionamento Ext. - Problema de saÃºde',
'Acionamento Ext. - Promessa de Pagamento',
'Acionamento Ext. - Sem CondiÃ§Ãµes Financeiras',
'Alega Pagamento',
'Alega Processo Judicial',
'ContestaÃ§Ã£o',
'EstÃ¡ negociando com o Credor',
'NÃ£o concorda com NegativaÃ§Ã£o SPC/SERASA',
'Recusa-se a pagar']

acionamentos_teles['CPC'] = np.where(acionamentos_teles['ACN_DESCRICAO'].isin(lista_cpc), 1, 0)
acionamentos_teles

,DEA_DATA,TIPO_PRODUTO,USU_NOME,DEV_CPF,ACN_DESCRICAO,Faixa_de_Atraso,ACIONAMENTO,CPC
19985,01/07/2024,Claro Móvel,VIVIAN ROSA OLIVEIRA BATISTA,19260658837,ACORDO,PDD (0 A 90),1,1
19995,01/07/2024,Claro Móvel,VIVIAN ROSA OLIVEIRA BATISTA,04475012406,ACORDO,Atraso (91 a 360),1,1
19997,01/07/2024,Claro Móvel,JORGE LUIS CARUSO,49119168349,VOLTAR A LIGAR,Atraso (361 a 1080),1,0
19998,01/07/2024,Claro Móvel,GILCILENE MENEGUITE,22121042865,LIGAÇÃO MUDA,Atraso (Acima 1801),1,0
19999,01/07/2024,Claro Móvel,LETICIA VIANNA RIBEIRO DE FREITAS,72898461920,RECADO COM TERCEIRO,PDD (0 A 90),1,0
...,...,...,...,...,...,...,...,...
82772,11/07/2024,GEVENUE NET,FELIPE DIAS ALVES LUIS,16942531472,Acionamento Ext. - Queda de LigaÃ§Ã£o,Atraso (1081 a 1800),1,0
82773,11/07/2024,GEVENUE CLARO MOVEL,NASCIMENTO VIANA RAMON,49500422115,Acionamento Ext. - Telefone Errado,Atraso (361 a 1080),1,0
82774,11/07/2024,GEVENUE NET,CRISTINA ASSUMPCAO DA PENHA RENATA,26513048591,Acionamento Ext. - Telefone Errado,Atraso (1081 a 1800),1,0
82775,11/07/2024,GEVENUE CLARO MOVEL,MOTA DA SILVA GABRIEL,02737398673,InformaÃ§Ãµes de contato - Adicionado,Atraso (361 a 1080),1,0


In [11]:
acionamentos_teles.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ACIONAMENTOS_TELEES/acionamentos_teles_julho.csv",index=False,sep=';',encoding='iso-8859-1')